In [1]:
from bs4 import BeautifulSoup
import requests
import pickle

import pandas as pd
import numpy as np

from functools import reduce
from tqdm.notebook import tqdm_notebook

from multiprocessing.pool import ThreadPool as Pool

import warnings
warnings.filterwarnings('ignore')

Read links

In [2]:
with open('data/osszes_link.pkl', 'rb') as pkl:
    osszes_link = pickle.load(pkl)

In [3]:
len(osszes_link)

80449

### Get data

In [4]:
valtozok = ['Alaptípus ára:', 'Extrákkal növelt ár:', 'Akciós ár:', 'Akció feltételei:', 'Vételár:', 'Vételár EUR:', 'Átvehető:', 'Évjárat:', 'Állapot:', 'Kivitel:', 'Garancia:',
            'Kilométeróra állása:', 'Szállítható szem. száma:', 'Ajtók száma:', 'Szín:', 'Saját tömeg:', 'Teljes tömeg:',
            'Csomagtartó:', 'Klíma fajtája:', 'Üzemanyag:', 'Hengerűrtartalom:', 'Teljesítmény:',
            'Hajtás:', 'Sebességváltó fajtája:', 'Okmányok jellege:', 'Műszaki vizsga érvényes:']

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [6]:
def scrape_car_listing(url):
    
    page = requests.get(url, headers = headers)
    
    if page.ok == False:
        pass
    
    soup = BeautifulSoup(page.text)
    
    adatlap = soup.find('div', id = 'adatlap')
    
    if len(adatlap) == 0:
        pass

    else:        
        hirdetes_nev = adatlap.find('div', class_ = 'adatlap-cim').select('h1')[0].text.strip()
        
        kiajanlo_adatok = soup.find_all('div', class_ = 'adatlap-kiajanlo-adatok')
        katalogus_adatok = [i for i in kiajanlo_adatok if 'Katalógus adatok' in str(i)]

        if len(katalogus_adatok) == 1:

            katalogus = katalogus_adatok[0]
            kat_nevek = [i.text for i in katalogus.find_all('div', class_ = 'col-xs-15')]
            kat_ertekek = [i.text for i in katalogus.find_all('div', class_ = 'col-xs-13')]

            katalogus_tabla = pd.DataFrame.from_dict(dict(zip(kat_nevek, kat_ertekek)), orient = 'index').reset_index().rename(columns = {'index' : 'variable', 0 : 'value'})

        else:
            katalogus_tabla = pd.DataFrame(columns = ['variable', 'value'])

        felszereltseg = adatlap.find('div', class_ = 'felszereltseg')        
        felszereltseg_tipus = [i.text for i in felszereltseg.find_all('h4')]
        felszereltseg_lista = [felszereltseg_tipus.find_all('li') for felszereltseg_tipus in felszereltseg.find_all('ul', class_ = 'pontos')]
        felszereltseg_lista = [[felszereltseg.text for felszereltseg in felszereltseg_tipus] for felszereltseg_tipus in felszereltseg_lista]
        felszereltseg_dict = dict(zip(felszereltseg_tipus, felszereltseg_lista))
        
        felszereltseg_dict = {'variable' : 'Felszereltség', 'value' : felszereltseg_dict}
        
        url_dict = {'variable' : 'URL', 'value' : url}
        nev_dict = {'variable' : 'Hirdetés név', 'value'  : hirdetes_nev}
        
        hirdetes_adatok = adatlap.find('table', class_ = 'hirdetesadatok')

        if hirdetes_adatok is None:            
            pass

        else:                
            adatok = pd.read_html(str(hirdetes_adatok))[0]            
            adatok.columns = ['variable', 'value']
            adatok = adatok.loc[adatok['variable'].isin(valtozok)]

            adatok = adatok.append(url_dict, ignore_index = True)
            adatok = adatok.append(nev_dict, ignore_index = True)
            adatok = adatok.append(felszereltseg_dict, ignore_index = True)
            adatok = adatok.append(katalogus_tabla, ignore_index = True)
            
    return adatok

Try running on 20 listings

In [12]:
%%time

collection = []

pool = Pool(8)

for link in tqdm_notebook(osszes_link[:20], desc = 'Collecting listings data into data frames'):
    pool.apply_async(scrape_car_listing, (link,), callback = collection.append)

pool.close()
pool.join()

Wall time: 3.84 s


20 listings was 3.9 seconds, so 80500 listings will be 80500 / 20 * 3.9 / 60 / 60 hours

In [13]:
80500 / 20 * 3.9 / 60 / 60

4.360416666666667

### Run scraper

and pray to God connection doesnt break for 4-5 hours...

Next time: dont append to list, save dataframes into ../data, and after all is finished, read all, merge all

In [16]:
len(osszes_link)

80449

In [17]:
%%time

collection = []

pool = Pool(8)

for link in tqdm_notebook(osszes_link, desc = 'Collecting listings data into data frames'):
    pool.apply_async(scrape_car_listing, (link,), callback = collection.append)

pool.close()
pool.join()

Wall time: 4h 3min 1s


In [18]:
len(collection)

74650

In [20]:
# with open('data/osszes_hirdetes.pkl', 'wb') as pkl:
#     pickle.dump(collection, pkl)

### Merge dataframes

In [ ]:
autok = reduce(lambda a, b: pd.merge(a, b, on = 'variable', how = 'outer'), collection).T
autok.reset_index(drop = True, inplace = True)
autok.columns = autok.iloc[0]
autok.drop(autok.index[0], inplace = True)

In [10]:
autok.head(3)

,Vételár:,Vételár EUR:,Évjárat:,Állapot:,Kivitel:,Kilométeróra állása:,Szállítható szem. száma:,Ajtók száma:,Saját tömeg:,Teljes tömeg:,...,Oktánszám,Városi fogyasztás,Országúti fogyasztás,Vegyes fogyasztás,CO2-kibocsátás,Végsebesség,Gyorsulás 0-ról 100 km/h-ra,Szín:,Műszaki vizsga érvényes:,Hasmagasság
1,370 000 Ft,€ 1 029,2004/1,Normál,Ferdehátú,275 656 km,5 fő,5,950 kg,1 445 kg,...,95,"7,3 l/100km","4,8 l/100km","5,7 l/100km",136 g/km,155 km/h,"14,3 s",NaN,NaN,NaN
2,2 369 000 Ft,€ 6 590,2013/10,Újszerű,Kombi,61 255 km,5 fő,5,1 105 kg,1 610 kg,...,95,7 l/100km,"4,7 l/100km","5,5 l/100km",127 g/km,167 km/h,"15,4 s",Ezüst (metál),2023/7,NaN
3,900 000 Ft,€ 2 504,2005/2,Normál,Kombi,305 000 km,5 fő,5,1 505 kg,2 080 kg,...,NaN,"7,7 l/100km","4,9 l/100km","5,9 l/100km",155 g/km,208 km/h,"9,8 s",Ezüst,2022/3,NaN


In [ ]:
autok.to_csv('data/autok.csv', index = False)

### Which links did I lose during scraping?

In [ ]:
np.setdiff1d(osszes_link, autok['URL'].tolist())